In [6]:
!wget https://hobbitdata.informatik.uni-leipzig.de/NCES_Ontolearn_Data/NCESFamilyData.zip -O ../NCESFamilyData.zip
!unzip -o -d ../ ../NCESFamilyData.zip
!rm -f ../NCESFamilyData.zip

--2023-08-16 11:53:00--  https://hobbitdata.informatik.uni-leipzig.de/NCES_Ontolearn_Data/NCESFamilyData.zip
Resolving hobbitdata.informatik.uni-leipzig.de (hobbitdata.informatik.uni-leipzig.de)... 139.18.121.22
Connecting to hobbitdata.informatik.uni-leipzig.de (hobbitdata.informatik.uni-leipzig.de)|139.18.121.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5334826 (5.1M) [application/zip]
Saving to: ‘../NCESFamilyData.zip’

../NCESFamilyData.z 100%[===================>]   5.09M  9.77MB/s    in 0.5s    

2023-08-16 11:53:01 (9.77 MB/s) - ‘../NCESFamilyData.zip’ saved [5334826/5334826]

Archive:  ../NCESFamilyData.zip
   creating: ../NCESData/family/
   creating: ../NCESData/family/embeddings/
  inflating: ../NCESData/family/embeddings/loss_per_epoch.csv  
  inflating: ../NCESData/family/embeddings/ConEx_relation_embeddings.csv  
  inflating: ../NCESData/family/embeddings/ConEx_entity_embeddings.csv  
  inflating: ../NCESData/family/embeddings/results.json 

In [1]:
from ontolearn.concept_learner import NCES
from ontolearn.knowledge_base import KnowledgeBase
from owlapy.parser import DLSyntaxParser
from owlapy.render import DLSyntaxObjectRenderer
import sys
from quality_functions import quality
import time

nces = NCES(knowledge_base_path="../NCESData/family/family.owl", learner_name="SetTransformer",\
            path_of_embeddings="../NCESData/family/embeddings/ConEx_entity_embeddings.csv",load_pretrained=True,\
            max_length=48, proj_dim=128, rnn_n_layers=2, drop_prob=0.1, num_heads=4, num_seeds=1, num_inds=32,\
            pretrained_model_name=["SetTransformer", "LSTM", "GRU"])

KB = KnowledgeBase(path=nces.knowledge_base_path)
dl_syntax_renderer = DLSyntaxObjectRenderer()
dl_parser = DLSyntaxParser(nces.kb_namespace)
brother = dl_parser.parse('Brother')
daughter = dl_parser.parse('Daughter')

pos = set(KB.individuals(brother)).union(set(KB.individuals(daughter)))
neg = set(KB.individuals())-set(pos)

t0 = time.time()
concept = nces.fit(pos, neg)
# Use NCES to synthesize the solution class expression.
# Note that NCES is not given the concepts Brother and Daughter.
# Yet, it is able to compute the exact solution!
t1 = time.time()
print("Duration: ", t1-t0, " seconds")
print("\nPrediction: ", dl_syntax_renderer.render(concept))
quality(KB, concept, pos, neg);



 Loaded pretrained model! 



 Loaded pretrained model! 



 Loaded pretrained model! 

Prediction:  Brother ⊔ Daughter
Duration:  0.3181018829345703  seconds

Prediction:  Brother ⊔ Daughter
Accuracy: 100.0%
Precision: 100.0%
Recall: 100.0%
F1: 100.0%
